In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import random
from tqdm import tqdm
import numpy as np
from sklearn.datasets.samples_generator import make_blobs
from sklearn.cluster import SpectralClustering, AffinityPropagation
import scipy
from scipy.sparse import csgraph

from numpy import linalg as LA
import utils
plt.ion()
plt.show()

In [ ]:
import matplotlib
import tools as t
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from scipy import spatial
from mpl_toolkits.mplot3d import Axes3D
from scipy.sparse.csgraph import connected_components
from sklearn.metrics import pairwise_kernels
from sklearn.metrics.pairwise import rbf_kernel

from networkx.algorithms.cuts import conductance
import networkx as nx 
sns.set_style('darkgrid')
%matplotlib inline

In [ ]:
data = np.load('/home/zz452/cluster/data/modelTishby_0_layer_data.p')
layer = data[-2] 
X = layer[40,:,:]
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(X[:,0],
           X[:,1], 
           X[:,2])
ax.title.set_text('Latent Visualizer')

#fig = plt.figure(figsize=(5, 5))
#plt.hist(M_d.flatten(), log=True, bins=100)
#plt.title('Distance Distribution')

# 2nd smallest eigenvalue : Fiedler value: bipartitioning graph

In [ ]:
perf = []
nb_clusters = 2

for idx in range(84):
    X = layer[idx,:,:]
    #clustering = SpectralClustering(n_clusters = nb_clusters, assign_labels="discretize", random_state=0).fit(X)
    #y_pred = clustering.labels_
    #perf.append(utils.internalValidation(X, y_pred)['_calinski_harabaz_score'])
    affinity_matrix = pairwise_kernels(X, metric = 'rbf')
    D = np.diag(affinity_matrix.sum(axis=1))
    L = D - affinity_matrix
    vals, vecs = np.linalg.eig(L)

    # sort
    #vecs = vecs[:,np.argsort(vals)]
    vals = vals[np.argsort(vals)]
    print(vals[1])
    # use Fiedler value to find best cut to separate data
    perf.append(vals[1]) 

# fig = plt.figure(figsize=(10, 10))
# ax = fig.add_subplot(111, projection='3d')
# ax.scatter(X[:,0],
#            X[:,1], 
#            X[:,2], c = y_pred)
# ax.title.set_text('Latent Visualizer')


# plt.figure(figsize=(14,6))
# plt.subplot(111)
# plt.title(f'Spectral clustering results ')
# plt.scatter(X[:, 0], X[:, 1], s=50, c = y_pred);



# Quality of clustering with silhouette coefficient

In [ ]:
perf = []
nb_clusters = 2

for idx in range(84):
    X = layer[idx,:,:]
    clustering = SpectralClustering(n_clusters = nb_clusters, assign_labels="discretize", random_state=0).fit(X)
    y_pred = clustering.labels_
    perf.append(utils.internalValidation(X, y_pred)['_silhouette_score'])

# Sample conductance estimation from graph

In [ ]:
affinity_matrix = pairwise_kernels(X, metric = 'rbf')
G=nx.from_numpy_matrix(affinity_matrix)
conductance(G, np.where(y_pred==0)[0], np.where(y_pred==1)[0])

# Eigenvalue Heuristic

In [ ]:

def eigenDecomposition(A, plot = True, topK = 5):
    """
    :param A: Affinity matrix
    :param plot: plots the sorted eigen values for visual inspection
    :return A tuple containing:
    - the optimal number of clusters by eigengap heuristic
    - all eigen values
    - all eigen vectors
    
    This method performs the eigen decomposition on a given affinity matrix,
    following the steps recommended in the paper:
    1. Construct the normalized affinity matrix: L = D−1/2ADˆ −1/2.
    2. Find the eigenvalues and their associated eigen vectors
    3. Identify the maximum gap which corresponds to the number of clusters
    by eigengap heuristic
    
    References:
    https://papers.nips.cc/paper/2619-self-tuning-spectral-clustering.pdf
    http://www.kyb.mpg.de/fileadmin/user_upload/files/publications/attachments/Luxburg07_tutorial_4488%5b0%5d.pdf
    """
    L = csgraph.laplacian(A, normed=True)
    n_components = A.shape[0]
    
    # LM parameter : Eigenvalues with largest magnitude (eigs, eigsh), that is, largest eigenvalues in 
    # the euclidean norm of complex numbers.
#     eigenvalues, eigenvectors = eigsh(L, k=n_components, which="LM", sigma=1.0, maxiter=5000)
    eigenvalues, eigenvectors = LA.eig(L)
    eigenvalues = np.sort(np.real(eigenvalues))
    if plot:
        plt.title('Largest eigen values of input matrix')
        plt.scatter(np.arange(len(eigenvalues)), eigenvalues)
        plt.grid()
        
    # Identify the optimal number of clusters as the index corresponding
    # to the larger gap between eigen values
    index_largest_gap = np.argsort(np.diff(eigenvalues))[::-1][:topK]
    nb_clusters = index_largest_gap + 1
        
    return nb_clusters, eigenvalues, eigenvectors

In [ ]:
# affinity_matrix = pairwise_kernels(X, metric = 'rbf')
affinity_matrix = rbf_kernel(X,  gamma=1)
k,  eigenvalues, eigenvectors = eigenDecomposition(affinity_matrix)
print(f'Optimal number of clusters {k}')

In [ ]:
f